In [ ]:
import sys, os

In [ ]:
from exllamav2 import(
    ExLlamaV2,
    ExLlamaV2Config,
    ExLlamaV2Cache,
    ExLlamaV2Cache_8bit,
    ExLlamaV2Tokenizer,
    model_init,
)

from exllamav2.generator import (
    ExLlamaV2StreamingGenerator,
    ExLlamaV2BaseGenerator,
    ExLlamaV2Sampler
)


In [ ]:
model_directory = "/home/cwinkler/oreilly/models/Mistral-7B-instruct-exl2"

In [ ]:
config = ExLlamaV2Config()
config.model_dir = model_directory
config.prepare()

model = ExLlamaV2(config)
print("Loading model: " + model_directory)

cache = ExLlamaV2Cache(model, lazy = True)
model.load_autosplit(cache)

In [ ]:
tokenizer = ExLlamaV2Tokenizer(config)

# Initialize generator

generator = ExLlamaV2StreamingGenerator(model, cache, tokenizer)

In [ ]:
settings = ExLlamaV2Sampler.Settings()
settings.temperature = 0.1
settings.top_k = 50
settings.top_p = 0.8
settings.token_repetition_penalty = 1.05
settings.disallow_tokens(tokenizer, [tokenizer.eos_token_id])

max_new_tokens = 600


In [ ]:
prompt = "Tell me about O'Reilly Safari!"

In [ ]:
input_ids = tokenizer.encode(prompt)
prompt_tokens = input_ids.shape[-1]

In [ ]:
generator.warmup()

In [ ]:
generator.set_stop_conditions([])
generator.begin_stream(input_ids, settings)

In [ ]:
%%time
generated_tokens = 0
while True:
    chunk, eos, _ = generator.stream()
    generated_tokens += 1
    print (chunk, end = "")
    sys.stdout.flush()
    if eos or generated_tokens == max_new_tokens: break

In [ ]:
!nvidia-smi